## Import Dependencies

In [38]:
import numpy as np
from keras import backend as K
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import vgg19

In [39]:
#!conda install pillow -y
from PIL import Image
print(Image.__file__)

/home/sachin/anaconda3/envs/tf-deep/lib/python3.6/site-packages/PIL/Image.py


In [40]:
folder_name = 'img/'
target_image_path = folder_name+'target.jpg'
style_reference_image_path = folder_name+'reference.jpg'

In [41]:
# import sys
# from PIL import Image
# sys.modules['Image'] = Image 

In [ ]:
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width*img_height/height)

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [ ]:
def deprocess_image(x):
    x[:,:,0] += 103.939
    x[:,:,1] += 116.779
    x[:,:,2] += 123.68
    x = x[:,:,::-1]
    x = np.clip(x,0,255).astype('uint8')
    return x

## Load the trained VGG19 model

In [ ]:
target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_image,
                             style_reference_image,
                             combination_image], axis=0)

model = vgg19.VGG19(input_tensor=input_tensor,
                   weights = 'imagenet',
                   include_top = False)

print('Model Loaded')

Model Loaded


## Content Loss

In [ ]:
def content_loss(base, combination):
    return K.sum(K.square(combination- base))

## Style Loss

In [ ]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x,(2,0,1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S-C))/(4.0 * (channels**2)*(size**2))

## Total Variational Loss

In [ ]:
def total_variation_loss(x):
    a = K.square(
        x[:,:img_height-1, :img_width-1, :] - 
        x[:,1:,:img_width-1,:])
    
    b = K.square(
        x[:,:img_height-1,:img_width-1,:] - 
        x[:,:img_height-1,1:,:])
    return K.sum(K.pow(a+b, 1.25))

In [ ]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = ['block1_conv1',
               'block2_conv1',
               'block3_conv1',
               'block4_conv1',
               'block5_conv1']

total_variation_weight = 1e-4
style_weight = 1.0
content_weight = 0.025

loss = K.variable(0.0)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0,:,:,:]
combination_features = layer_features[2,:,:,:]

loss += content_weight * content_loss(target_image_features, combination_features)

for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1,:,:,:]
    combination_features = layer_features[2,:,:,:]
    s1 = style_loss(style_reference_features, combination_features)
    loss += (style_weight/ len(style_layers)) *s1
    
loss += total_variation_weight * total_variation_loss(combination_image)


In [ ]:
grads = K.gradients(loss, combination_image)[0]
fetch_loss_and_grads = K.function([combination_image],[loss, grads])

class Evaluator(object):
    
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
        
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_value = None
        return grad_values

In [ ]:
evaluator = Evaluator()

In [ ]:
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

In [ ]:
result_prefix = folder_name+'my_result'
iterations = 50

In [ ]:
x = preprocess_image(target_image_path)
x = x.flatten()

for i in range(iterations):
    print('Start of iteration: ',i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss,x,fprime=evaluator.grads, maxfun=20)
    print('Current loss value: ', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png'%i
    imsave(fname, img)
    print('Image saved as: ', fname)
    end_time = time.time()
    print('Iteraion %d completed in %ds '% (i, end_time-start_time))

Start of iteration:  0
Current loss value:  1374335700.0
Image saved as:  img/my_result_at_iteration_0.png
Iteraion 0 completed in 5s 
Start of iteration:  1


/home/sachin/anaconda3/envs/tf-deep/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if sys.path[0] == '':


Current loss value:  500177440.0
Image saved as:  img/my_result_at_iteration_1.png
Iteraion 1 completed in 5s 
Start of iteration:  2
Current loss value:  279504160.0
Image saved as:  img/my_result_at_iteration_2.png
Iteraion 2 completed in 5s 
Start of iteration:  3
Current loss value:  186898260.0
Image saved as:  img/my_result_at_iteration_3.png
Iteraion 3 completed in 5s 
Start of iteration:  4
Current loss value:  144537070.0
Image saved as:  img/my_result_at_iteration_4.png
Iteraion 4 completed in 5s 
Start of iteration:  5
Current loss value:  120775240.0
Image saved as:  img/my_result_at_iteration_5.png
Iteraion 5 completed in 5s 
Start of iteration:  6
Current loss value:  106237840.0
Image saved as:  img/my_result_at_iteration_6.png
Iteraion 6 completed in 5s 
Start of iteration:  7
Current loss value:  94279176.0
Image saved as:  img/my_result_at_iteration_7.png
Iteraion 7 completed in 5s 
Start of iteration:  8
Current loss value:  86003560.0
Image saved as:  img/my_result_

In [ ]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))